In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np


In [ ]:

# Load the pre-trained model
model = torch.load('path_to_model')
model.eval()  # Set the model to evaluation mode


In [ ]:

# Define transformations to apply to each test image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model's expected input size
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

test_folder = 'path_to_your_test_folder'

# Function to predict class for a single image
def predict_image(image_path, model, transform):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
    _, predicted = torch.max(output, 1)
    return predicted.item()  # Return the predicted class index

# List to store true and predicted labels
y_true = []
y_pred = []

# Iterate over images in the test folder
for filename in os.listdir(test_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(test_folder, filename)
        true_label = int(filename.split('_')[0])  # Assuming filename is like "class_label_filename.jpg"
        predicted_label = predict_image(image_path, model, transform)
        y_true.append(true_label)
        y_pred.append(predicted_label)

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)


In [ ]:

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
cm = confusion_matrix(y_true, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:")
print(cm)
